In [1]:
# 导入工具包
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
# 数据读取
test_data = pd.read_csv('./data_format1/test_format1.csv')
train_data = pd.read_csv('./data_format1/train_format1.csv')
user_info = pd.read_csv('./data_format1/user_info_format1.csv')   # 特征表
user_log = pd.read_csv('./data_format1/user_log_format1.csv')     # 日志信息

In [3]:
print(train_data.columns)
print(test_data.columns)
print(user_info.columns)
print(user_log.columns)

Index(['user_id', 'merchant_id', 'label'], dtype='object')
Index(['user_id', 'merchant_id', 'prob'], dtype='object')
Index(['user_id', 'age_range', 'gender'], dtype='object')
Index(['user_id', 'item_id', 'cat_id', 'seller_id', 'brand_id', 'time_stamp',
       'action_type'],
      dtype='object')


In [4]:
print(train_data.shape)
print(test_data.shape)
print(user_info.shape)
print(user_log.shape)

(260864, 3)
(261477, 3)
(424170, 3)
(54925330, 7)


In [5]:
print(train_data.info())
print('*'*50)
print(test_data.info())
print('*'*50)
print(user_info.info())   # 特征表有缺失
print('*'*50)
print(user_log.info())    # 日志有缺失

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260864 entries, 0 to 260863
Data columns (total 3 columns):
user_id        260864 non-null int64
merchant_id    260864 non-null int64
label          260864 non-null int64
dtypes: int64(3)
memory usage: 6.0 MB
None
**************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261477 entries, 0 to 261476
Data columns (total 3 columns):
user_id        261477 non-null int64
merchant_id    261477 non-null int64
prob           0 non-null float64
dtypes: float64(1), int64(2)
memory usage: 6.0 MB
None
**************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424170 entries, 0 to 424169
Data columns (total 3 columns):
user_id      424170 non-null int64
age_range    421953 non-null float64
gender       417734 non-null float64
dtypes: float64(2), int64(1)
memory usage: 9.7 MB
None
**************************************************
<class 'pandas.core.frame.DataFrame'

In [6]:
column1 = ['age_range','gender']
column2 = ['cat_id','seller_id','brand_id','time_stamp','action_type']
for column in column1:
    print(column,user_info[column].isnull().sum()/user_info.shape[0])
print('='*100)
for column in column2:
    print(column,user_log[column].isnull().sum()/user_log.shape[0])

age_range 0.005226677982884221
gender 0.01517316170403376
cat_id 0.0
seller_id 0.0
brand_id 0.0016570678774255885
time_stamp 0.0
action_type 0.0


In [7]:
print(user_info.groupby(['age_range'])[['user_id']].count())
print(user_info.groupby(['gender'])[['user_id']].count())
print(train_data.groupby(['label'])[['user_id']].count())

           user_id
age_range         
0.0          92914
1.0             24
2.0          52871
3.0         111654
4.0          79991
5.0          40777
6.0          35464
7.0           6992
8.0           1266
        user_id
gender         
0.0      285638
1.0      121670
2.0       10426
       user_id
label         
0       244912
1        15952


# 数据合并、预处理

In [8]:
train = train_data.copy()
test = test_data.copy()
info = user_info.copy()  # 特征
log = user_log.copy()   # 日志

In [9]:
print(train.shape)
print(test.shape)
print(info.shape)
print(log.shape)

(260864, 3)
(261477, 3)
(424170, 3)
(54925330, 7)


In [10]:
test.drop('prob',axis=1,inplace=True)

In [11]:
all_data = train.append(test)

In [12]:
print(all_data.shape)

(522341, 3)


In [13]:
all_data.head()

,label,merchant_id,user_id
0,0.0,3906,34176
1,0.0,121,34176
2,1.0,4356,34176
3,0.0,2217,34176
4,0.0,4818,230784


In [14]:
all_data = all_data.merge(info,on=['user_id'],how='left')  # 以左边的user_id为基准合并，右边如果没有用NaN填补

print(all_data.shape)

(522341, 5)


In [15]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 522341 entries, 0 to 522340
Data columns (total 5 columns):
label          260864 non-null float64
merchant_id    522341 non-null int64
user_id        522341 non-null int64
age_range      519763 non-null float64
gender         514796 non-null float64
dtypes: float64(3), int64(2)
memory usage: 23.9 MB


## 处理行为日志

In [16]:
list_join_func = lambda x:" ".join([str(i) for i in x])

agg_dict = {'item_id':list_join_func,
           'cat_id':list_join_func,
           'seller_id':list_join_func,
           'brand_id':list_join_func,
           'time_stamp':list_join_func,
           'action_type':list_join_func}
rename_dict = {'item_id' : 'item_path',
            'cat_id' : 'cat_path',
            'seller_id' : 'seller_path',
            'brand_id' : 'brand_path',
            'time_stamp' : 'time_stamp_path',
            'action_type' : 'action_type_path'}

In [17]:
log_path1 = log.groupby(['user_id']).agg(agg_dict).reset_index().rename(columns=rename_dict)

In [18]:
log_path1.head()

,user_id,item_path,cat_path,seller_path,brand_path,time_stamp_path,action_type_path
0,1,112203 556107 411984 112203 112203 112203 1110...,1252 1252 1252 1252 1252 1252 992 992 992 992 ...,4026 739 4177 4026 4026 4026 1019 1019 1019 10...,1469.0 6822.0 1960.0 1469.0 1469.0 1469.0 6805...,1018 1018 1018 1018 1021 1021 1111 1111 1111 1...,0 0 0 0 2 0 0 0 0 0 0 2 2 2 2 0 0 0 0 0 0 0 0 ...
1,2,239288 751744 431502 864941 85520 765891 41934...,602 602 737 737 737 737 737 737 737 602 1213 6...,420 420 1974 1974 1974 1974 1974 1974 1974 420...,4953.0 4953.0 8321.0 8321.0 8321.0 8321.0 8321...,626 626 817 817 817 817 817 817 817 626 626 62...,0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 3 ...
2,3,863889 828013 734838 909738 203050 763658 6641...,662 662 656 662 606 656 1467 656 1134 1134 113...,2372 139 947 883 2313 947 947 2957 4461 4461 4...,7862.0 3228.0 305.0 1647.0 3322.0 305.0 305.0 ...,921 921 1111 915 1111 1111 1111 1111 820 819 8...,0 0 0 0 2 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 ...
3,4,1019133 185328 606202 1019133 830140 836727 24...,748 748 748 748 748 612 1577 1505 1505 1505 15...,434 1607 1607 434 1607 1221 1221 1221 1221 122...,8010.0 3843.0 3843.0 8010.0 3843.0 7734.0 7734...,1101 1101 1101 1101 1101 527 527 527 527 527 5...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4,5,597305 99491 698879 996987 52951 941292 597305...,1577 1658 407 737 1326 389 1577 1147 407 407 4...,2676 186 3163 3969 3067 414 2676 1961 3163 316...,3997.0 7591.0 3881.0 483.0 4744.0 4299.0 3997....,1111 1111 1111 1111 1111 1111 1111 1111 1111 1...,0 0 0 0 0 0 0 0 2 2 2 0 0 0 2 2 0 0 0 0 0 0 0 ...


In [19]:
log_path1.shape

(424170, 7)

In [20]:
log_path1 = log_path1.sort_values(['user_id'])

# 这是合并后的完整数据

In [21]:
all_data_path = all_data.merge(log_path1,on='user_id')

In [22]:
all_data_path.shape

(522341, 11)

In [23]:
all_data_path.head()

,label,merchant_id,user_id,age_range,gender,item_path,cat_path,seller_path,brand_path,time_stamp_path,action_type_path
0,0.0,3906,34176,6.0,0.0,790010 231901 478025 100290 655967 986148 8547...,1467 662 662 662 420 276 1553 1238 821 1208 82...,4231 1945 3700 2179 1875 3001 1815 331 3906 43...,3589.0 2758.0 2204.0 6741.0 2714.0 5738.0 7169...,1026 818 818 818 1110 1110 1110 1110 1110 1110...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1,0.0,121,34176,6.0,0.0,790010 231901 478025 100290 655967 986148 8547...,1467 662 662 662 420 276 1553 1238 821 1208 82...,4231 1945 3700 2179 1875 3001 1815 331 3906 43...,3589.0 2758.0 2204.0 6741.0 2714.0 5738.0 7169...,1026 818 818 818 1110 1110 1110 1110 1110 1110...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2,1.0,4356,34176,6.0,0.0,790010 231901 478025 100290 655967 986148 8547...,1467 662 662 662 420 276 1553 1238 821 1208 82...,4231 1945 3700 2179 1875 3001 1815 331 3906 43...,3589.0 2758.0 2204.0 6741.0 2714.0 5738.0 7169...,1026 818 818 818 1110 1110 1110 1110 1110 1110...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
3,0.0,2217,34176,6.0,0.0,790010 231901 478025 100290 655967 986148 8547...,1467 662 662 662 420 276 1553 1238 821 1208 82...,4231 1945 3700 2179 1875 3001 1815 331 3906 43...,3589.0 2758.0 2204.0 6741.0 2714.0 5738.0 7169...,1026 818 818 818 1110 1110 1110 1110 1110 1110...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4,0.0,4818,230784,0.0,0.0,430968 185046 316409 713880 498279 503629 9596...,115 737 407 407 407 407 407 407 407 407 407 11...,376 3904 3556 3556 3556 3556 3556 3556 3556 35...,4167.0 3876.0 1236.0 1236.0 1236.0 1236.0 1236...,701 622 630 630 630 630 630 630 630 630 630 70...,0 0 0 0 0 0 0 0 0 0 2 0 2 0 2 2 0 2 2 0 0 0 0 ...


In [24]:
all_data_path.isnull().sum()

label               261477
merchant_id              0
user_id                  0
age_range             2578
gender                7545
item_path                0
cat_path                 0
seller_path              0
brand_path               0
time_stamp_path          0
action_type_path         0
dtype: int64

# 特征工程

In [25]:
all_data1 = all_data_path.copy()    # "user_id"第一份完整的数据

In [26]:
# 构造特征的函数
# ------------------------------------------------------
# 统计数据不去重
def cnt_(x):
    try:
        return len(x.split(' '))
    except:
        return -1
def user_cnt(df_data,single_col,name):
    df_data[name] = df_data[single_col].apply(cnt_)
    return df_data
# ------------------------------------------------------
# 统计数据并去重
def nunique_(x):
    try:
        return len(set(x.split(' ')))
    except:
        return -1
def user_nunique(df_data,single_col,name):
    df_data[name] = df_data[single_col].apply(nunique_)
    return df_data
# --------------------------------------------------------
# 统计用户最早操作时间
def _min(x):
    try:
        return np.min([int(i) for i in x.split(' ')])
    except:
        return -1
def user_min(df_data,single_col,name):
    df_data[name] = df_data[single_col].apply(_min)
    return df_data
# 统计用户最晚操作时间
def _max(x):
    try:
        return np.max([int(i) for i in x.split(' ')])
    except:
        return -1
def user_max(df_data,single_col,name):
    df_data[name] = df_data[single_col].apply(_max)
    return df_data
# 统计用户最早最晚操作时间差
def _std(x):
    try:
        return np.std([float(i) for i in x.split(' ')])
    except:
        return -1
def user_std(df_data,single_col,name):
    df_data[name] = df_data[single_col].apply(_std)
    return df_data
# 统计用户操作最早和最晚相差天数
def int_to_datatime(x):
    x = str(x)
    x = '2020-'+x[:-2]+'-'+x[-2:]
    return pd.to_datatime(x,errors='ignore')
def time_range(df_data, max_date_col,min_date_col,name):
    max_date = list(map(int_to_datetime,copy.deepcopy(df_data[max_date_col])))
    min_date = list(map(int_to_datetime,copy.deepcopy(df_data[min_date_col])))
    ns_to_day = 1e9*60*60*24
    ans = []
    for i in range(len(max_date)):
        ans.append((max_date[i]-min_date[i]).value/ns_to_day)
    df_data[name] = ans
    return df_data

# 统计topN的数据
def most_n(x,n):
    try:
        return Counter(x.split(' ')).most_common(n)[n-1][0]
    except:
        return -1
def user_most_n(df_data,single_col,name,n=1):
    func = lambda x:most_n(x,n)
    df_data[name] = df_data[single_col].apply(func)
    return df_data
# 统计topN数据的总数
def most_n_cnt(x,n):
    try:
        return Counter(x.split(' ')).most_common(n)[n-1][1]
    except:
        return -1
def user_most_n_cnt(df_data,single_col,name,n=1):
    func = lambda x:most_n_cnt(x,n)
    df_data[name] = df_data[single_col].apply(func)
    return df_data
# 统计每个用户不同行为的次数
def user_action_cnt(df_data,col_action,action_type,name):
    func = lambda x:len([i for i in x.split(' ') if i == action_type])
    df_data[name+'_'+action_type] = df_data[col_action].apply(func)
    return df_data
# 用户针对此商家 之前的打分
def user_merchant_mark(df_data,merchant_id,seller_path,action_type_path,action_type):
    seller_len = len(df_data[seller_path].split(' '))
    data_dict = {}
    data_dict[seller_path] = df_data[seller_path].split(' ')             # 键是商家店铺，值是商家店铺id列表
    data_dict[action_type_path] = df_data[action_type_path].split(' ')   # 键是用户行为，值是用户行为方式列表
    mark = 0
    for i_ in range(seller_len):
        if data_dict[seller_path][i_] == str(df_data[merchant_id]):      # 对应商家
            if data_dict[action_type_path][i_] == action_type:           # 对应行为
                mark += 1
    return mark
def user_merchant_mark_all(df_data,merchant_id,seller_path,action_type_path,action_type,name):
    df_data[name+'_'+action_type] = df_data.apply(lambda x:user_merchant_mark(x,merchant_id,seller_path,action_type_path,action_type),axis=1)
    return df_data


In [27]:
# 统计用户特征
# 1.用户对多少种物品进行操作
all_data1 = user_nunique(all_data1,'item_path','user_item_counts')
# 2,用户对多少种类目进行操作
all_data1 = user_nunique(all_data1,'cat_path','user_item_counts')

# 3.用户总共对多少个商店进行过操作(不去重)
all_data1 = user_cnt(all_data1,'seller_path','user_seller_counts')
all_data1 = user_cnt(all_data1,'seller_path','user_seller_unique_counts')
# 4.用户对多少个品牌进行了操作
all_data1 = user_nunique(all_data1,'brand_path','user_brand_counts')
# 5.用户活跃的天数
all_data1 = user_nunique(all_data1,'time_stamp_path','user_day_activity_counts')
# 6.用户有几种行为
all_data1 = user_nunique(all_data1,'action_type_path','user_action_type_counts')
#----------------------------------------------------------------------------
# 7.用户最早操作时间
all_data1 = user_min(all_data1,'time_stamp_path','user_time_stamp_min')
# 8.用户最晚操作时间
all_data1 = user_max(all_data1,'time_stamp_path','user_time_stamp_max')
# 9.用户最早最晚操作标准差
all_data1 = user_std(all_data1,'time_stamp_path','user_time_stamp_std')
# 10.统计用户最早最晚操作相差的天数
#all_data1 = user_range(all_data1,'user_time_stamp_max','user_time_stamp_min','user_time_stamp_range')
# ------------------------------------------------------------------------------
# 用户最喜欢操作的类目，点击、加购物车、分享收藏
all_data1 = user_most_n(all_data1,'cat_path','user_cat_most_1',n = 1)
all_data1 = user_most_n_cnt(all_data1,'cat_path','user_cat_most_1_cnt',n=1)
# 用户最喜欢的店铺，操作次数
all_data1 = user_most_n(all_data1,'seller_path','user_seller_most_1',n=1)
all_data1 = user_most_n_cnt(all_data1,'seller_path','user_seller_most_1_cnt',n=1)
# 用户最喜欢的品牌，操作次数
all_data1 = user_most_n(all_data1,'brand_path','user_brand_most_1',n=1)
all_data1 = user_most_n_cnt(all_data1,'brand_path','user_brand_most_1_cnt',n=1)
# 用户最喜欢的操作方式，操作次数
all_data1 = user_most_n(all_data1,'action_type_path','action_most_1',n=1)
all_data1 = user_most_n_cnt(all_data1,'action_type_path','action_most_1_cnt',n=1)
# -----------------------------------------------------------------------------------
# 统计用户点击、加入购物车、购买、收藏的次数
all_data1 = user_action_cnt(all_data1,'action_type_path','0','user_cnt')
all_data1 = user_action_cnt(all_data1,'action_type_path','1','user_cnt')
all_data1 = user_action_cnt(all_data1,'action_type_path','2','user_cnt')
all_data1 = user_action_cnt(all_data1,'action_type_path','3','user_cnt')
# ---------------------------------------------------------------------------------
# 对性别和年龄进行独热编码
age_range = pd.get_dummies(all_data1['age_range'],prefix='age')
gender_range = pd.get_dummies(all_data1['gender'],prefix='gender')
all_data1 = all_data1.join(age_range)
all_data1 = all_data1.join(gender_range)


In [28]:
all_data11 = all_data1.copy()

# 店铺特征统计

In [31]:
list_join_func = lambda x:' '.join([str(i) for i in x])
agg_dict_seller = {
    'user_id':list_join_func,
    'item_id':list_join_func,
    'cat_id':list_join_func,
    'brand_id':list_join_func,
    'time_stamp':list_join_func,
    'action_type':list_join_func
}
rename_dict_seller = {
            'user_id':'user_path',
            'item_id' : 'item_path',
            'cat_id' : 'cat_path',
            'brand_id' : 'brand_path',
            'time_stamp' : 'time_stamp_path',
            'action_type' : 'action_type_path'  
}
log_path_2 = log.groupby('seller_id').agg(agg_dict_seller).reset_index().rename(columns=rename_dict_seller)

In [44]:
log_path_2.to_csv('log_path_2')

In [47]:
# all_data2 = all_data.merge(log_path_2,on='seller_id')

In [ ]:
all_data2.to_csv('all_data2')

# all_data_2是以seller_id为基准合并后的完整的数据集

In [32]:
# 店铺统计特征
all_data2_test = log_path_2.copy()

In [33]:
# 1.商店被操作的总次数
all_data2_test = user_cnt(all_data2_test,'user_path','seller_user_counts')
# 2.商店被不同用户操作的次数
all_data2_test = user_nunique(all_data2_test,'user_path','seller_user_unique_counts')
# 3.商店中有多少种不同的商品
all_data2_test = user_nunique(all_data2_test,'item_path','seller_item_unique_counts')
# 4.商店中有多少类不同过得商品
all_data2_test = user_nunique(all_data2_test,'cat_path','seller_cat_unique_counts')
# 5.商店中有多少不同的品牌
all_data2_test = user_nunique(all_data2_test,'brand_path','seller_brand_unique_counts')
# 6.商店中活跃的天数
all_data2_test = user_nunique(all_data2_test,'time_stamp_path','seller_day_active_counts')

# 7.最晚时间
all_data2_test = user_max(all_data2_test,'time_stamp_path','seller_time_stamp_max')
# 8.最早时间
all_data2_test = user_min(all_data2_test,'time_stamp_path','seller_time_stamp_min')
# 9.活跃天数标准差
all_data2_test = user_std(all_data2_test,'time_stamp_path','seller_time_stamp_std')
# 10.最早和最晚相差时间
# all_data2_test = user_range(all_data2_test,'seller_time_stamp_max','seller_time_stamp_min','seller_time_stamp_range_day')

# 11.统计商店被点击的次数之和
all_data2_test = user_action_cnt(all_data2_test,'action_type_path','0','seller_cnt')
all_data2_test = user_action_cnt(all_data2_test,'action_type_path','1','seller_cnt')
all_data2_test = user_action_cnt(all_data2_test,'action_type_path','2','seller_cnt')
all_data2_test = user_action_cnt(all_data2_test,'action_type_path','3','seller_cnt')


# 购买两次以上的买家总数

In [34]:
all_data2_test.shape

(4995, 20)

In [ ]:
all_data1.shape

# 用户+商家综合特征

In [35]:
# 用户对此商家有多少次0,1,2,3动作
all_data1 = user_merchant_mark_all(all_data1,'merchant_id','seller_path','action_type_path','0','user_merchant_action')
all_data1 = user_merchant_mark_all(all_data1,'merchant_id','seller_path','action_type_path','1','user_merchant_action')
all_data1 = user_merchant_mark_all(all_data1,'merchant_id','seller_path','action_type_path','2','user_merchant_action')
all_data1 = user_merchant_mark_all(all_data1,'merchant_id','seller_path','action_type_path','3','user_merchant_action')


In [36]:
all_data1.columns

Index(['label', 'merchant_id', 'user_id', 'age_range', 'gender', 'item_path',
       'cat_path', 'seller_path', 'brand_path', 'time_stamp_path',
       'action_type_path', 'user_item_counts', 'user_seller_counts',
       'user_seller_unique_counts', 'user_brand_counts',
       'user_day_activity_counts', 'user_action_type_counts',
       'user_time_stamp_min', 'user_time_stamp_max', 'user_time_stamp_std',
       'user_cat_most_1', 'user_cat_most_1_cnt', 'user_seller_most_1',
       'user_seller_most_1_cnt', 'user_brand_most_1', 'user_brand_most_1_cnt',
       'action_most_1', 'action_most_1_cnt', 'user_cnt_0', 'user_cnt_1',
       'user_cnt_2', 'user_cnt_3', 'age_0.0', 'age_1.0', 'age_2.0', 'age_3.0',
       'age_4.0', 'age_5.0', 'age_6.0', 'age_7.0', 'age_8.0', 'gender_0.0',
       'gender_1.0', 'gender_2.0', 'user_merchant_action_0',
       'user_merchant_action_1', 'user_merchant_action_2',
       'user_merchant_action_3'],
      dtype='object')

In [37]:
all_data2_test.columns

Index(['seller_id', 'user_path', 'item_path', 'cat_path', 'brand_path',
       'time_stamp_path', 'action_type_path', 'seller_user_counts',
       'seller_user_unique_counts', 'seller_item_unique_counts',
       'seller_cat_unique_counts', 'seller_brand_unique_counts',
       'seller_day_active_counts', 'seller_time_stamp_max',
       'seller_time_stamp_min', 'seller_time_stamp_std', 'seller_cnt_0',
       'seller_cnt_1', 'seller_cnt_2', 'seller_cnt_3'],
      dtype='object')

In [38]:
all_data2_test = all_data2_test.rename(columns={'seller_id':'merchant_id'})

In [ ]:
# 1.用户在商铺购买商品总数

# 2.用户在商铺购买商铺类别数目
# 3.用户在商铺中购买品牌总数

# 4.用户在商铺中出现的月份
# 5.用户在商铺中出现的天数
# 6.用户在商铺中每月访问的次数

# 8.用户在商铺中点击、加购物车、购买、收藏数量统计

# 将构建好的特征进行合并

In [39]:
all_data3 = all_data1.merge(all_data2_test,on='merchant_id',how='left')

In [66]:
all_data3.isnull().sum()

label                         261477
merchant_id                        0
user_id                            0
age_range                       2578
gender                          7545
item_path_x                        0
cat_path_x                         0
seller_path                        0
brand_path_x                       0
time_stamp_path_x                  0
action_type_path_x                 0
user_item_counts                   0
user_seller_counts                 0
user_seller_unique_counts          0
user_brand_counts                  0
user_day_activity_counts           0
user_action_type_counts            0
user_time_stamp_min                0
user_time_stamp_max                0
user_time_stamp_std                0
user_cat_most_1                    0
user_cat_most_1_cnt                0
user_seller_most_1                 0
user_seller_most_1_cnt             0
user_brand_most_1                  0
user_brand_most_1_cnt              0
action_most_1                      0
a

In [67]:
all_data3.to_csv('all_data3')

OSError: [Errno 28] No space left on device

# all_data3即就是最终的特征

## 利用词袋模型统计特征

In [69]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from scipy import sparse

tfidfVec = TfidfVectorizer(stop_words = ENGLISH_STOP_WORDS,
                          ngram_range=(1,1),
                          max_features=100)
columns_list = ['seller_path','item_path','brand_path','time_stamp_path','action_type_path']
for i,col in enumerate(columns_list):
    tfidfVec.fit(all_data3[col])
    data_ = tfidfVec.transform(all_data3[col])
    if i == 0:
        data_cat = data_
    else:
        data_cat = sparse.hstack((data_cat,data_))

KeyError: 'item_path'

In [ ]:
df_tfidf = pd.DataFrame(data_cat.toarray())
df_tfidf.columns = ['tfidf_'+ str(i) for i in df_tfidf.columns]
all_data4 = pd.concat([all_data3,df_tfidf],axis=1)

#### 这里的all_data即就是综合了用户特征、商家特征、用户+商家特征、词袋模型提取特征

In [ ]:
all_data4.shape

In [ ]:
all_data4.columns

# 接下来进行一般的数据探索

In [30]:
all_data11.columns

Index(['label', 'merchant_id', 'user_id', 'age_range', 'gender', 'item_path',
       'cat_path', 'seller_path', 'brand_path', 'time_stamp_path',
       'action_type_path', 'user_item_counts', 'user_seller_counts',
       'user_seller_unique_counts', 'user_brand_counts',
       'user_day_activity_counts', 'user_action_type_counts',
       'user_time_stamp_min', 'user_time_stamp_max', 'user_time_stamp_std',
       'user_cat_most_1', 'user_cat_most_1_cnt', 'user_seller_most_1',
       'user_seller_most_1_cnt', 'user_brand_most_1', 'user_brand_most_1_cnt',
       'action_most_1', 'action_most_1_cnt', 'user_cnt_0', 'user_cnt_1',
       'user_cnt_2', 'user_cnt_3', 'age_0.0', 'age_1.0', 'age_2.0', 'age_3.0',
       'age_4.0', 'age_5.0', 'age_6.0', 'age_7.0', 'age_8.0', 'gender_0.0',
       'gender_1.0', 'gender_2.0'],
      dtype='object')

In [31]:
delete_features = ['item_path','cat_path', 'seller_path', 'brand_path', 'time_stamp_path','action_type_path']
all_data5 = all_data11.copy()
all_data5.drop(delete_features,axis=1,inplace=True)
all_data5.head()

,label,merchant_id,user_id,age_range,gender,user_item_counts,user_seller_counts,user_seller_unique_counts,user_brand_counts,user_day_activity_counts,...,age_2.0,age_3.0,age_4.0,age_5.0,age_6.0,age_7.0,age_8.0,gender_0.0,gender_1.0,gender_2.0
0,0.0,3906,34176,6.0,0.0,45,451,451,108,47,...,0,0,0,0,1,0,0,1,0,0
1,0.0,121,34176,6.0,0.0,45,451,451,108,47,...,0,0,0,0,1,0,0,1,0,0
2,1.0,4356,34176,6.0,0.0,45,451,451,108,47,...,0,0,0,0,1,0,0,1,0,0
3,0.0,2217,34176,6.0,0.0,45,451,451,108,47,...,0,0,0,0,1,0,0,1,0,0
4,0.0,4818,230784,0.0,0.0,17,54,54,19,16,...,0,0,0,0,0,0,0,1,0,0


In [32]:
all_data5['label'].values

array([ 0.,  0.,  1., ..., nan, nan, nan])

In [33]:
train_data = all_data5.loc[all_data5['label'] >= 0]
print(train_data.shape)

(260864, 38)


In [34]:
# train_data = all_data5[all_data5['label'] >= 0]  # 训练 数据
test_data = all_data5.iloc[260864:]   # 测试数据
# print(train_data.shape)
print(test_data.shape)

(261477, 38)


In [56]:
train_data['gender'].value_counts()

0.0    176414
1.0     73756
2.0      6983
Name: gender, dtype: int64

In [57]:
train_data['age_range'].value_counts()

3.0    69369
0.0    55809
4.0    51235
2.0    31026
5.0    25618
6.0    21701
7.0     4120
8.0      720
1.0       13
Name: age_range, dtype: int64

In [35]:
from sklearn.impute import SimpleImputer 
imp_mode = SimpleImputer(strategy='most_frequent')

In [36]:
for column in ['age_range','gender']:
    train_data[column] = imp_mode.fit_transform(train_data[column].values.reshape(-1,1))

# 至此，得到全部数据all_data5,未对其进行stacking操作，训练数据train_data,测试数据test_data

# 下面进行数据探索，模型训练

这里，因为原始数据都是强相关的，不再进行KDE查看了，只查看数据间的相关性，因为数据也没有多少，所以不进行降维或特征选择了，直接对此模型在训练集和测试集上进行
训练和验证即可，全部性能都只由特征工程支撑   
在这里，构造特征之前没有进行缺失值处理，没有进行异常值处理，构造的特征将会更加偏离原始特征，第二次操作之前要对其进行操作，这样更加贴合实际。

In [37]:
train_X = train_data[:10000].copy()
test_X = test_data[:1000].copy()
print(train_X.shape)
print(test_X.shape)

(10000, 38)
(1000, 38)


In [41]:
test_data.drop('label',axis=1,inplace=True)
test_X.drop('label',axis=1,inplace=True)
print(test_X.shape)

(1000, 37)


In [42]:
label = train_X['label'].values      # 标签label

train_X.drop('label',axis=1,inplace=True)
print(train_X.shape)

(10000, 37)


In [105]:
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.metrics import roc_auc_score

clf = RFC(n_estimators=100,max_depth=2,random_state=0)

X_train,X_test,y_train,y_test = train_test_split(train_X,label,test_size=0.4,random_state=0)

clf = clf.fit(X_train,y_train)

In [64]:
score = cross_val_score(clf,train_X,label,cv=5).mean()
score2 = cross_val_score(clf,train_X,label,cv=5,scoring='f1_macro').mean()

In [64]:
print('单个score Accuracy：',clf.score(X_test,y_test))
print('CV_mean_score Accuracy:',score)
print("AUC:",roc_auc_score(y_test,clf.predict(X_test)))
print("F1:",score2)

ImportError: cannot import name 'trian_test_split' from 'sklearn.model_selection' (C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\__init__.py)

In [48]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
stdScaler = StandardScaler()
X = stdScaler.fit_transform(train_X)
X_train,X_test,y_train,y_test = train_test_split(X,label,test_size=0.4,random_state=0)
clf = LogisticRegression(random_state=0,solver='lbfgs',multi_class='multinomial').fit(X_train,y_train)
clf.score(X_test,y_test)   # R方

0.93375

In [52]:
from sklearn import tree
from sklearn.ensemble import BaggingClassifier as BC,RandomForestClassifier as RFC,ExtraTreesClassifier as ETC
from sklearn.ensemble import AdaBoostClassifier as ABC,GradientBoostingClassifier as GBC
import lightgbm as lgb
from sklearn.neighbors import KNeighborsClassifier as KN

In [53]:
X_train,X_test,y_train,y_test = train_test_split(train_X,label,test_size=0.4,random_state=0)
clf1 = tree.DecisionTreeClassifier()
clf2 = BC(KN(),max_samples = 0.5,max_features=0.5)
clf3 = RFC(n_estimators=10)
clf4 = ETC(n_estimators=10)
clf5 = ABC(n_estimators=10)
clf6 = GBC(n_estimators=10)
# clf7 = lgb
for clf in [clf1,clf2,clf3,clf4,clf5,clf6]:
    clf = clf.fit(X_train,y_train)
    print(clf.score(X_test,y_test))

0.8745
0.9335
0.9275
0.92375
0.9335
0.9335


逻辑回归、K近邻、线性回归、贝叶斯这些模型用之前要进行标准化。而树模型，决策树、随机森林、集成模型GBDT、XGBoost、Lightgbm等用之前不用进行标准化操作。